### 진짜 완전 새롭게 다시 시작

In [1]:
import gymnasium as gym
import horcrux_terrain_v2
from horcrux_terrain_v2.envs import PlaneJoyWorld
import numpy as np

from ray.tune.registry import register_env
from ray.rllib.algorithms.sac import SACConfig

MJCF Model Path: 
c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml


In [2]:
env_config = {
    "forward_reward_weight": 6.5,
    "side_cost_weight": 2.0,
    "unhealthy_max_steps": 100,
    "healthy_reward": 0.5,
    "healthy_roll_range": (-35,35),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 0.01,
    "rotation_orientation_cost_weight": 1.2,
    "termination_reward": 0,
    "gait_params": (30, 30, 60, 60, 0),
    "use_friction_chg": True,
    "joy_input_random": True,
}

# JoyWorld
register_env("joy-v1", lambda env_config: PlaneJoyWorld(env_config))


In [ ]:
config = (
    SACConfig()
    .environment("joy-v1")
    .env_runners(num_env_runners=1)
    .training(
        gamma=0.9,
        actor_lr=0.001,
        critic_lr=0.002,
        train_batch_size_per_learner=256,
    )
)
# Build the SAC algo object from the config and run 1 training iteration.
algo = config.build_algo()
algo.train()

2025-03-13 17:51:56,461	WARNING algorithm_config.py:4703 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-03-13 17:51:56,461	WARNING sac.py:487 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
c:\Users\doore\anaconda3\envs\gd\Lib\site-packages\ray\rllib\algorithms\algorithm.py:512: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You coul

(SingleAgentEnvRunner pid=32352) MJCF Model Path: 
(SingleAgentEnvRunner pid=32352) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml
(SingleAgentEnvRunner pid=2924) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml
(SingleAgentEnvRunner pid=32216) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml
(SingleAgentEnvRunner pid=19968) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml
(SingleAgentEnvRunner pid=33936) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml
(SingleAgentEnvRunner pid=31088) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml
(SingleA

2025-03-13 17:52:08,867	ERROR actor_manager.py:833 -- Ray error (The actor died because of an error raised in its creation task, ray::SingleAgentEnvRunner.__init__() (pid=32352, ip=127.0.0.1, actor_id=d80c73bc0c2524ec5a387e4f01000000, repr=<ray.rllib.env.single_agent_env_runner.SingleAgentEnvRunner object at 0x000001B990129610>)
  File "python\\ray\\_raylet.pyx", line 1883, in ray._raylet.execute_task
  File "python\\ray\\_raylet.pyx", line 1824, in ray._raylet.execute_task.function_executor
  File "c:\Users\doore\anaconda3\envs\gd\Lib\site-packages\ray\_private\function_manager.py", line 696, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\doore\anaconda3\envs\gd\Lib\site-packages\ray\util\tracing\tracing_helper.py", line 463, in _resume_span
    return method(self, *_args, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\doore\anaconda3\envs\gd\Lib\site-packages\ray\rllib\e

(SingleAgentEnvRunner pid=21988) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml
(SingleAgentEnvRunner pid=9732) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml
(SingleAgentEnvRunner pid=2864) c:\users\doore\documents\project\snake_rl\gd_tor\learning\horcrux_terrain_v2\horcrux_terrain_v2\resources\horcrux_plane.xml


AttributeError: 'EnvContext' object has no attribute 'startswith'